In [1]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('C:/Users/Sharwari Prakash M/Dropbox/My PC (DESKTOP-48H0BCO)/Downloads/dt.csv')
df.head()

# Define the calculate entropy function
def calculate_entropy(df_label):
    classes, class_counts = np.unique(df_label, return_counts=True)
    entropy_value = np.sum(
        [(-class_counts[i] / np.sum(class_counts)) * np.log2(class_counts[i] / np.sum(class_counts)) for i in
         range(len(classes))])
    return entropy_value

# Define the calculate information gain function
def calculate_information_gain(dataset, feature, label):
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])
    values, feat_counts = np.unique(dataset[feature], return_counts=True)

    # Calculate the weighted feature entropy
    weighted_feature_entropy = np.sum(
        [(feat_counts[i] / np.sum(feat_counts)) * calculate_entropy(
            dataset.where(dataset[feature] == values[i]).dropna()[label]) for i in range(len(values))])
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

# Define the create decision tree function
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])

    if len(unique_data) <= 1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]

    elif len(features) == 0:
        return parent

    else:
        parent = unique_data[np.argmax(datum[1])]

        # Call the calculate_information_gain function
        item_values = [calculate_information_gain(dataset, feature, label) for feature in features]

        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature: {}}

        features = [i for i in features if i != optimum_feature]

        for value in np.unique(dataset[optimum_feature]):
            min_data = dataset.where(dataset[optimum_feature] == value).dropna()

            # Recursive call to create_decision_tree function
            min_tree = create_decision_tree(min_data, df, features, label, parent)

            decision_tree[optimum_feature][value] = min_tree

        return decision_tree

# Define the predict function
def predict_diabetes(test_data, decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        if value not in decision_tree[nodes]:
            return "Cannot predict"

        decision_tree = decision_tree[nodes][value]

        if type(decision_tree) is dict:
            # Recursive call to predict_diabetes function
            prediction = predict_diabetes(test_data, decision_tree)
        else:
            prediction = decision_tree
            break

    return prediction

# Set the features and label
features = df.columns[:-1]
label = 'diabetes'
parent = None

# Train the decision tree model
decision_tree = create_decision_tree(df, df, features, label, parent)

# Predict using the trained model
sample_data = {'glucose': 86, 'bloodpressure': 104}
test_data = pd.Series(sample_data)

prediction = predict_diabetes(test_data, decision_tree)
print(prediction)


1.0
